In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.metrics import precision_score, recall_score
import csv
import tldextract
import urllib.parse
import re



/Users/greglamex/anaconda3/envs/scikit-learn/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
def clean_url(url):
    URL = 'http://' + url if re.match(r'^(?:http|ftp)s?://', url) is None else url
    return URL



In [3]:
# 数点
def count_dots(url):
    return url.count('.')



In [4]:
def url_length(url):
    return len(str(url))




In [5]:

# 是否存在顶级域名
def has_subdomain(url):
    return 1 if tldextract.extract(url)[0] is not None else 0

In [6]:
# 求出domain的长度
def domain_lentgh(url):
    return len(tldextract.extract(url)[1])



In [7]:
# 求出域名后缀的长度
def suffix_length(url):
    return len(tldextract.extract(url)[2])




In [8]:
# 是否存有路径
def has_path(url):
    return 1 if urllib.parse.urlparse(url)[2] is not None else 0



In [9]:
# 是否查询context
def has_query(url):
    return 1 if urllib.parse.urlparse(url)[4] is not None else 0




In [10]:
# 计算路径长度
def path_length(url):
    return len(urllib.parse.urlparse(url)[2])



In [11]:
# 计算查询内容长度
def query_length(url):
    return len(urllib.parse.urlparse(url)[4])




In [12]:
# 数/
def count_SubDir(url):
    return url.count('/')



In [13]:
# 数@
def count_At(url):
    return url.count('@')




In [14]:
# 放弃
def has_IP(url):
    compile_rule = re.compile(r'(?<![\.\d])(?:\d{1,3}\.){3}\d{1,3}(?![\.\d])')
    match_list = re.findall(compile_rule, url)
    if match_list:
        return 1
    else:
        return 0

In [15]:
def get_features(temp, url):
    temp.append(count_dots(url))
    temp.append(url_length(url))
    temp.append(has_subdomain(url))
    temp.append(has_path(url))
    temp.append(has_query(url))
    temp.append(path_length(url))
    temp.append(query_length(url))
    temp.append(domain_lentgh(url))
    temp.append(suffix_length(url))
    temp.append(count_SubDir(url))
    temp.append(has_IP(url))
    temp.append(count_At(url))
    return temp



In [16]:
if __name__ == '__main__':
    url = []
    label = []
    temp = []
    data = []
    LaBel = []

    with open(r'data.csv', 'r', encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            url.append(clean_url(row['url']))
            if row['label'] == 'good':
                label.append(0)
            elif row['label'] == 'bad':
                label.append(1)
        csvfile.close()
    # print(label[:10])
    # print(countdots(url[1]))
    for i in range(0, len(url)):
        temp = get_features(temp, url[i])
        data.append(temp)
        LaBel.append(label[i])
        temp = []

    data = np.array(data)
    LaBel = np.array(LaBel)

    X_train, X_test, y_train, y_test = train_test_split(data, LaBel, test_size=0.3, random_state=0)
    sc = StandardScaler()

    sc.fit(X_train)
    # 对数据集进行处理
    url_tree = tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2,
                                           min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=10,
                                           random_state=None, max_leaf_nodes=None,
                                           min_impurity_decrease=0.0, min_impurity_split=None, presort=False)

    print("***************start****************")
    url_tree_save = url_tree.fit(X_train, y_train)
    print("**************saveing***************")
    # joblib.dump(url_tree_save, 'C:/Users/Birdman/Desktop/data/url_tree.model')
    print("Training score:%f" % (url_tree.score(X_train, y_train)))
    print("Testing score:%f" % (url_tree.score(X_test, y_test)))
    ''' 
    clf = RandomForestClassifier()
    clf.fit(X_train,y_train)
    print("Training score:%f" % (clf.score(X_train, y_train)))
    print("Testing score:%f" % (clf.score(X_test, y_test)))
    '''

    y_pred = url_tree.predict(X_test)
    print("Training precision score:%f" % (precision_score(y_test, y_pred)))
    print("Training recall score:%f" % (recall_score(y_test, y_pred)))
    # test

    url = "www.baidu.com"
    test = []
    test = np.array(get_features(test, url))
    if url_tree.predict([test]) == 1:
        print("test result:  GOOD")
    else:
        print("test result:  BAD")
    '''if clf.predict([test]) == 1:
        print("test result:  GOOD")
    else:
        print("test result:  BAD")'''

***************start****************
**************saveing***************
Training score:0.948513
Testing score:0.909252
Training precision score:0.796647
Training recall score:0.667721
test result:  BAD
